# Sentinel-3 Snow and ICE products (SICE)

In [1]:
#%run ./prepare-sice-environment.ipynb
! pip install --upgrade sentinelhub 

Defaulting to user installation because normal site-packages is not writeable


In [1]:

## Credentials needs to be filled in the code in order to make it work!

import matplotlib.pyplot as plt
import numpy as np
import os
import logging
import subprocess
from botocore.client import Config as botoConfig
from botocore.exceptions import ClientError
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import boto3
import glob
import time
import rasterio
from shapely import geometry
from osgeo import ogr

logging = logging.getLogger(__name__)

def plot_image(image, factor=1.0, clip_range=None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])
    
    
def merge_tiffs(input_filename_list, merged_filename, *, overwrite=False, delete_input=False):
    """Performs gdal_merge on a set of given geotiff images

    :param input_filename_list: A list of input tiff image filenames
    :param merged_filename: Filename of merged tiff image
    :param overwrite: If True overwrite the output (merged) file if it exists
    :param delete_input: If True input images will be deleted at the end
    """
    if os.path.exists(merged_filename):
        if overwrite:
            os.remove(merged_filename)
        else:
            raise OSError(f"{merged_filename} exists!")

    logging.info("merging %d tiffs to %s", len(input_filename_list), merged_filename)
    subprocess.check_call(
        ["gdal_merge.py", "-co", "BIGTIFF=YES", "-co", "compress=LZW", "-ot", "Float32" , "-init", "-9999" , "-n", "-9999", "-a_nodata", "-9999", "-o", merged_filename, *input_filename_list]
    )
    logging.info("merging done")

    if delete_input:
        logging.info("deleting input files")
        for filename in input_filename_list:
            if os.path.isfile(filename):
                os.remove(filename)
                
def deleteProcessingData(dir):
    logging.info(f'Deleting files from {dir}')
    for file in glob.glob(f'{dir}/*_tmp.tif'):
        logging.info(f'Deleting file {file}')
        os.remove(file)

def importToBucket(awsConfig, resultsFolder, processBands=False):
    
    try:
        s3_client = boto3.resource('s3',
                                   endpoint_url=awsConfig['s3Url'],
                                   use_ssl=False,
                                   aws_access_key_id=awsConfig['s3AccessKey'],
                                   aws_secret_access_key=awsConfig['s3SecrestKey'],
                                   config=botoConfig(
                                       signature_version='s3',
                                       connect_timeout=60,
                                       read_timeout=60,
                                   ))

        bucket = s3_client.Bucket(awsConfig['bucketName'])
    except Exception as e:
        logging.error(e)
        raise Exception('Invalid bucket parameters') 
    
    coverGeometry = ''
    
    # get tif files to upload
    filenamesList = glob.glob(f'{resultsFolder}/*.tif')
    try:
        for file in filenamesList:
            destFile = file.replace(resultsFolder, awsConfig['folder'])
            if processBands:
                tmpFile = file.replace(".tif", "_tmp.tif")
                cmd = "gdal_translate -b 1 -of COG -a_nodata -9999 -co COMPRESS=DEFLATE -co BLOCKSIZE=1024 -co RESAMPLING=AVERAGE -co OVERVIEWS=IGNORE_EXISTING {0} {1}".format(file, tmpFile)
                logging.info(cmd)
                os.system(cmd)
                file = tmpFile
                
            if not coverGeometry:
                coverGeometry = getMbrWithIntermediate(file)
            
            logging.info(f'Uploading {file} to {destFile}')
            response = bucket.upload_file(file, destFile)
        deleteProcessingData(resultsFolder)
    except Exception as e:
        logging.error(e)
        raise Exception(f'Failed to upload file: {destFile}') 
        
    return coverGeometry

def addIntermediateY(x, y, diff, numPoints):
    coords = []
    coords.append([x, y])  
    for i in range(numPoints):
        coords.append([x, y + i / numPoints * diff])
    return coords

def addIntermediateX(x, y, diff, numPoints):
    coords = []
    coords.append([x, y])  
    for i in range(numPoints):
        coords.append([x + i / numPoints * diff, y])
    return coords

def getMbrWithIntermediate(file):
    dataset = rasterio.open(file)
    bounds = dataset.bounds
    poly = getPolygonFromMbr(bounds.left, bounds.bottom, bounds.right, bounds.top, 20)
    p = geometry.mapping(poly)
    p['properties'] = {'name': 'urn:ogc:def:crs:EPSG::4326'}
    return p         
        
def getPolygonFromMbr(xMin, yMin, xMax, yMax, numPoints):
    coords = []
    xDiff = xMax - xMin
    yDiff = yMax - yMin
    
    coords.extend(addIntermediateY(xMin, yMin, yDiff, numPoints))
    coords.extend(addIntermediateX(xMin, yMax, xDiff, numPoints))
    coords.extend(addIntermediateY(xMax, yMax, -yDiff, numPoints))
    coords.extend(addIntermediateX(xMax, yMin, -xDiff, numPoints))
    poly = geometry.Polygon(coords)
    return poly        
        
def deleteTile(s3folder, oauth, byocUrl):
    url = f'{byocUrl}/tiles'

    try:
        while url is not None:
            print(f'Calling url: {url}')
            response = oauth.get(url)
            response.raise_for_status()

            output = response.json()
            tiles = output['data']
            links = output['links']

            for tile in tiles:
                if tile['path'].startswith(s3folder):
                    tile_id = tile['id']
                    response = oauth.delete(f'{byocUrl}/tiles/{tile_id}')
                    print("Deleted tile: " + tile['path'])
                    try:
                        response.raise_for_status()
                    except Exception as e:
                        print("Failed to delete tile {0}: {1}".format(
                            tile_id, response.reason))
                        logging.error(e)

            # sets url to None if there's no link to the next set of tiles
            url = links.get('next', None)

            # waits a bit before fetching the next set
            time.sleep(0.1)
    except Exception as e:
        logging.error("BYOC delete error: {0}".format(response.reason))
        logging.error(e)
        raise

def insertTile(tile, oauth, byocUrl):
    print(f'Inserting tile {tile}')
    try:
        response = oauth.post(f'{byocUrl}/tiles', json=tile)
        response.raise_for_status()
    except Exception as e:
        logging.error("BYOC import error: {0}".format(response.reason))
        logging.error(e)


def importToBYOC(config):
     # Create a session
    client = BackendApplicationClient(client_id=config['client_id'])
    oauth = OAuth2Session(client=client)
    oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                      client_id=['client_id'], client_secret=config['client_secret'])

    byoc_service_base_url = config['byoc_service_base_url']
    collection_id = config['collection_id']
    byocUrl = f'{byoc_service_base_url}/collections/{collection_id}'
    
    # ingest tile
    tilePath = config['folder']
    tile = {
        'path': tilePath,
        'sensingTime': config['sensingTime'],
        'coverGeometry' : config['coverGeometry']
    }
    
    if config['coverGeometry']: 
         tile ['coverGeometry'] = config['coverGeometry']
            
    logging.info("Deleting folder: " + tilePath)
    deleteTile(tilePath, oauth, byocUrl)
    logging.info("Ingesting folder: " + tilePath)
    insertTile(tile, oauth, byocUrl)
    
    
def defaultBYOCImport(resultsFolder, s3Folder, resolution, sensingTime):
    if not (resolution == 300 or resolution == 500) :
        logger.info('Resolution not supported.')
        return
    
    s3config = {
        's3Url' : 'https://s3.waw2-1.cloudferro.com',
        's3AccessKey' : 'xxx',
        's3SecrestKey' : 'xxx',
        'bucketName' : 'polar',
        'folder' : s3Folder,
    }
    coverGeometry = importToBucket(s3config, resultsFolder, True)
    
    collectionID_300m = 'd34c470c-52a8-49db-9f9b-0956f10734d9'
    collectionID_500m = '6a3a4f71-84ff-4421-95a7-6ba969b5cf88'
    
    collectionId = collectionID_300m if resolution == 300 else collectionID_500m
    
    shConfig = {
        'byoc_service_base_url' : 'https://creodias.sentinel-hub.com/api/v1/byoc',
        'client_id' : 'xxx',
        'client_secret' : 'xxx',
        'collection_id' : collectionId,
        'folder' : s3Folder,
        'sensingTime' : sensingTime,
        'coverGeometry' : coverGeometry
    }    
    importToBYOC(shConfig)

In [3]:
# Various utilities
import os
import json
from shapely import geometry, wkt
import datetime as dt
import numpy as np
import geopandas as gpd
import glob
import shutil
import logging
import concurrent.futures
import time

from sentinelhub import SentinelHubDownloadClient, SentinelHubBatch, SentinelHubRequest, Geometry, CRS, DataCollection, MimeType, SHConfig, BBox, bbox_to_dimensions
from shapely.geometry import Polygon
import tarfile

# create logs folder
if not os.path.exists("logs"):
    os.makedirs("logs")

# right now we only log to consol
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler(f'logs/sice_sh_{time.strftime("%Y_%m_%d",time.localtime())}.log'),
        logging.StreamHandler()
    ])

In [4]:
#External variables
# Set the date of calculation
date = "2021-07-09"

# resolution (m) 
resolution = 1200  # minimum resolution of data is 300m

#area of interest
aoi = 'POLYGON((-53.6565 82.4951, -59.9608 82.1309, -67.7892 80.5602, -67.9606 80.0218, -67.6072 79.3014, -72.7375 78.5894, -73.5413 78.1636, -72.9428 77.3837, -69.0700 76.0128, -66.6509 75.7624, -60.3956 75.8231, -58.4311 74.8854, -55.1967 69.6980, -53.8565 68.8368, -54.2986 67.0754, -53.5562 65.6109, -52.3863 64.7989, -52.3228 64.0074, -50.2076 62.1010, -48.6300 60.7381, -45.0522 59.7674, -43.2890 59.6436, -42.4957 60.3093, -41.8486 61.5655, -41.6969 62.6486, -40.1106 63.5452, -39.9111 64.7944, -38.0777 65.4068, -36.9899 65.1987, -31.2165 67.7166, -25.8502 68.6303, -21.6517 70.0839, -20.9932 70.7880, -21.2829 72.9254, -16.9050 74.9601, -17.1213 79.6158, -10.2883 81.4244, -14.0398 81.9745, -17.8112 82.0131, -28.5252 83.7013, -40.1075 83.6651, -53.6565 82.4951))'
aoi = "POLYGON ((-14.675905 65.792421, -14.675905 66.238873, -13.423464 66.238873, -13.423464 65.792421, -14.675905 65.792421))"
#sub-area
#aoi = 'POLYGON ((-56.045995 66.271911, -49.369389 66.271911, -49.369389 71.949832, -56.045995 71.949832, -56.045995 66.271911))'

#Island
#aoi = 'POLYGON ((-24.634498 66.588207, -13.38895 66.588207, -13.38895 63.292939, -24.634498 63.292939, -24.634498 66.588207))'

#target projection of the final results
projection = '4326' #default

#projection = '3413'  #polar
 

# log processing parameters - don't log any S3 information
logging.info(f'Date: {date}')
logging.info(f'AOI: {aoi}')
logging.info(f'Projection: {projection}')



2023-03-07 14:49:11 [INFO] root - Date: 2021-07-09
2023-03-07 14:49:11 [INFO] root - AOI: POLYGON ((-14.675905 65.792421, -14.675905 66.238873, -13.423464 66.238873, -13.423464 65.792421, -14.675905 65.792421))
2023-03-07 14:49:11 [INFO] root - Projection: 4326


In [5]:
#verify input parameters

if not date:
    raise Exception('variable date has to be set') 

if not aoi:
    raise Exception('aoi has to be set')     

if not resolution:
    raise Exception('resolution has to be set')     

if resolution < 300 or resolution > 1200: 
    raise Exception('value of resolution has to be between 200 m and 1200 m')

#transform period into single day    
if '/' in date:
    date = date[0:date.find('/')]    

In [6]:
#system settings

#base folder where the code is located
USR_PATH = os.path.abspath('.')
EVAL_SCRIPT_PATH = os.path.join(USR_PATH, 'data_fusion_olci_dem.js') 

#delete download folder after processing - will save space but will download all the data for each requwst (otherwise the cached tile data will be used)
DELETE_DOWNLOAD_FOLDER = True

#OUTPUT_DIR = os.path.join(USR_PATH, "output") #local folder
OUTPUT_DIR = "/home/jovyan/result-data" # will be copied to the local folder of the user requesting the data

logging.info("System configuration ok.")    


2023-03-07 14:49:11 [INFO] root - System configuration ok.


In [7]:
# initial ENV configuration
SH_CLIENT_ID = %env SH_CLIENT_ID
SH_CLIENT_SECRET = %env SH_CLIENT_SECRET

sh_config = SHConfig()
sh_config.sh_base_url = "https://services.sentinel-hub.com"
sh_config.download_timeout_seconds=300
sh_config.download_sleep_time=20

sh_config.sh_client_id = SH_CLIENT_ID
sh_config.sh_client_secret = SH_CLIENT_SECRET

sh_config.save()

# Evalscript
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        datasource: "OLCI",
        bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10", "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B20", "B21", "SZA", "VZA", "SAA", "VAA", "TOTAL_COLUMN_OZONE"],
      },
      {
        datasource: "COP_30",
        bands: ["DEM"]
      }
    ],
    output: [
      {
        id: "r_TOA_01",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "r_TOA_06",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "r_TOA_17",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "r_TOA_21",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "snow_grain_diameter",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "snow_specific_surface_area",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "diagnostic_retrieval",
        bands: 1,
        sampleType: "UINT8",
      },
      {
        id: "albedo_bb_planar_sw",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "albedo_bb_spherical_sw",
        bands: 1,
        sampleType: "FLOAT32",
      }

    ],
    mosaicking: "SIMPLE",
  };
}

//function updateOutput(outputs, collections) {
//  Object.values(outputs).forEach((output) => {
//    output.bands = collections.scenes.length;
//  });
//}

// Set constants as global variables which can be used in all functions
var wls = {
  "B01": 0.4000E+00,
  "B02": 0.4125E+00,
  "B03": 0.4425E+00,
  "B04": 0.4900E+00,
  "B05": 0.5100E+00,
  "B06": 0.5600E+00,
  "B07": 0.6200E+00,
  "B08": 0.6650E+00,
  "B09": 0.6737E+00,
  "B10": 0.6812E+00,
  "B11": 0.7088E+00,
  "B12": 0.7538E+00,
  "B13": 0.7613E+00,
  "B14": 0.7644E+00,
  "B15": 0.7675E+00,
  "B16": 0.7788E+00,
  "B17": 0.8650E+00,
  "B18": 0.8850E+00,
  "B19": 0.9000E+00,
  "B20": 0.9400E+00,
  "B21": 0.1020E+01
};

var bai = {
  "B01": 2.365E-11,
  "B02": 2.7E-11,
  "B03": 7.0E-11,
  "B04": 4.17E-10,
  "B05": 8.04E-10,
  "B06": 2.84E-09,
  "B07": 8.58E-09,
  "B08": 1.78E-08,
  "B09": 1.95E-08,
  "B10": 2.1E-08,
  "B11": 3.3E-08,
  "B12": 6.23E-08,
  "B13": 7.1E-08,
  "B14": 7.68E-08,
  "B15": 8.13E-08,
  "B16": 9.88E-08,
  "B17": 2.4E-07,
  "B18": 3.64E-07,
  "B19": 4.2E-07,
  "B20": 5.53e-07,
  "B21": 2.25E-06
}

let emptyBandObject = {
  "B01": {},
  "B02": {},
  "B03": {},
  "B04": {},
  "B05": {},
  "B06": {},
  "B07": {},
  "B08": {},
  "B09": {},
  "B10": {},
  "B11": {},
  "B12": {},
  "B13": {},
  "B14": {},
  "B15": {},
  "B16": {},
  "B17": {},
  "B18": {},
  "B19": {},
  "B20": {},
  "B21": {}
}

const allBands = Object.keys(bai);

// solar spectrum constants
const f0 = 32.38;
const f1 = -160140.33;
const f2 = 7959.53;
const bet = 1. / (85.34 * 1.e-3);
const gam = 1. / (401.79 * 1.e-3);

var coef1, coef2 = analyt_func(0.3, 0.7);
var coef3, coef4 = analyt_func(0.7, 0.865);

function evaluatePixel(samples) {
  const n_acquisitions = samples.OLCI.length;
  const olciSamples = samples.OLCI;
  const demSamples = samples.COP_30;
  let r_TOA_01_valid = [];
  let r_TOA_06_valid = [];
  let r_TOA_17_valid = [];
  let r_TOA_21_valid = [];
  let snow_grain_diameter = [];
  let snow_specific_surface_area = [];

  // new products
  let diagnostic_retrieval = [];
  let albedo_bb_planar_sw = [];
  let albedo_bb_spherical_sw = [];



  // Loop through acquisition dates. No need to use this for single acquisition processing.
  for (aq = 0; aq < n_acquisitions; aq++) {
    // Correct TOA reflectance for ozone absorption
    sample_cor_o3 = ozone_correction(olciSamples[aq]);

    //Transfer of OLCI relative azimuthal angle to the definition used in radiative transfer code
    angles = view_geometry(olciSamples[aq]);

    //Filtering pixels unsuitable for retrieval
    [snow, sample_fltr] = prepare_processing(sample_cor_o3, demSamples[0]);

    //Compute snow properties
    [sample_valid, angles_valid, snow] = snow_properties(sample_fltr, angles, snow);

    //Compute aerosol
    aerosol = aerosol_properties(demSamples[0].DEM, angles_valid.cos_sa);

    //Compute atmosphere
    atmosphere = prepare_coef(aerosol, angles);

    //Compute theoretical reflectance of snow from albedo
    snow = clean_snow_albedo(sample_valid, angles, aerosol, atmosphere, snow);

    //throw new Error(JSON.stringify(snow))

    snow = polluted_snow_albedo(sample_valid, angles, aerosol, atmosphere, snow);

    snow = compute_plane_albedo(snow, angles);

    r_TOA_01_valid.push(sample_valid.B01);
    r_TOA_06_valid.push(sample_valid.B06);
    r_TOA_17_valid.push(sample_valid.B17);
    r_TOA_21_valid.push(sample_valid.B21);
    snow_grain_diameter.push(snow.diameter);
    snow_specific_surface_area.push(snow.area);
    diagnostic_retrieval.push(snow.isnow);
    albedo_bb_planar_sw.push(snow.rp3);
    albedo_bb_spherical_sw.push(snow.rs3);

    // Development use. Allow you to check the output before the script is finished.
    //throw new Error(JSON.stringify(snow))
  }
  return {
    r_TOA_01: r_TOA_01_valid,
    r_TOA_06: r_TOA_06_valid,
    r_TOA_17: r_TOA_17_valid,
    r_TOA_21: r_TOA_21_valid,
    snow_grain_diameter: snow_grain_diameter,
    snow_specific_surface_area: snow_specific_surface_area,
    diagnostic_retrieval: diagnostic_retrieval,
    albedo_bb_planar_sw: albedo_bb_planar_sw,
    albedo_bb_spherical_sw: albedo_bb_spherical_sw
  };
}

function deg2rad(deg) {
  const pi = Math.PI;
  return deg * (pi / 180);
}

function ozone_correction(sample) {
  const tozone_dict = {
    "B01": 1.378170469E-004,
    "B02": 3.048780958E-004,
    "B03": 1.645714060E-003,
    "B04": 8.935947110E-003,
    "B05": 1.750535146E-002,
    "B06": 4.347104369E-002,
    "B07": 4.487130794E-002,
    "B08": 2.101591797E-002,
    "B09": 1.716230955E-002,
    "B10": 1.466298300E-002,
    "B11": 7.983028470E-003,
    "B12": 3.879744653E-003,
    "B13": 2.923775641E-003,
    "B14": 2.792211429E-003,
    "B15": 2.729651478E-003,
    "B16": 3.255969698E-003,
    "B17": 8.956858078E-004,
    "B18": 5.188799343E-004,
    "B19": 6.715773241E-004,
    "B20": 3.127781417E-004,
    "B21": 1.408798425E-005
  };
  let toa_cor_o3 = {};
  // Loop through OLCI bands
  allBands.forEach(band => {
    todadu = sample.TOTAL_COLUMN_OZONE * 46696.24;
    inv_cos_za = 1 / Math.cos(deg2rad(sample.SZA)) + 1 / Math.cos(deg2rad(sample.VZA));
    toa_cor_o3[band] = sample[band] * 1 * Math.exp(inv_cos_za * tozone_dict[band] * todadu / 404.59);
  });

  toa_cor_o3.SZA = sample.SZA;
  toa_cor_o3.VZA = sample.VZA;
  toa_cor_o3.SAA = sample.SAA;
  toa_cor_o3.VAA = sample.VAA;
  return toa_cor_o3;
}

function view_geometry(sample) {
  const raa = 180 - (sample.VAA - sample.SAA);
  const sin_sza = Math.sin(deg2rad(sample.SZA));
  const sin_vza = Math.sin(deg2rad(sample.VZA));
  const cos_sza = Math.cos(deg2rad(sample.SZA));
  const cos_vza = Math.cos(deg2rad(sample.VZA));
  const ak1 = 3 * (1 + 2 * cos_sza) / 7;
  const ak2 = 3 * (1 + 2 * cos_vza) / 7;
  const cos_raa = Math.cos(deg2rad(raa));
  const inv_cos_za = 1 / cos_sza + 1 / cos_vza;
  const cos_sa = -cos_sza * cos_vza + sin_sza * sin_vza * cos_raa;
  return {
    raa: raa,
    cos_sza: cos_sza,
    cos_vza: cos_vza,
    ak1: ak1,
    ak2: ak2,
    inv_cos_za: inv_cos_za,
    cos_sa: cos_sa
  };
}

function prepare_processing(sample_cor, sample_dem) {
  let snow = {};
  let sample_fltr = {};

  // Assign diagnostic code
  snow.isnow = sample_cor.B21 < 0.1 ? 102 : NaN;
  snow.isnow = sample_cor.SZA > 75 ? 100 : snow.isnow;
  const mask = isNaN(snow.isnow);

  // Loop through OLCI bands
  allBands.forEach(band => {
    sample_fltr[band] = (mask == true) ? sample_cor[band] : NaN;
  });

  // Add elevation info to the object
  sample_fltr.elevation = (mask == true) ? sample_dem.DEM : NaN;
  return [
    snow,
    sample_fltr
  ];
}

function snow_properties(sample_fltr, angles, snow) {
  const akap2 = 2.25e-6;
  const alpha2 = 4 * Math.PI * akap2 / 1.020;
  const eps = 1.549559365010611;
  const rr1 = sample_fltr.B17;
  const rr2 = sample_fltr.B21;
  const ak1 = angles.ak1;
  const ak2 = angles.ak2;
  const r0 = Math.pow(rr1, eps) * Math.pow(rr2, (1 - eps));
  const bal = Math.pow((Math.log(rr2 / r0) / (ak1 * ak2 / r0)), 2) / alpha2;
  const al = bal / 1000;
  const D = al / (9.2 * 16 / 9);
  const area = 6 / D / 0.917;

  //Filtering small D
  const diameter_thresh = 0.01;
  const valid = D >= diameter_thresh;
  snow.isnow = (!valid && isNaN(snow.isnow)) ? 104 : snow.isnow;

  //Loop through toa bands
  let sample_valid = {};

  allBands.forEach(band => {
    sample_valid[band] = valid ? sample_fltr[band] : NaN;
  });

  //Assign valid snow properties
  snow.diameter = valid ? D : NaN;
  snow.area = valid ? area : NaN;
  snow.al = valid ? al : NaN;
  snow.r0 = valid ? r0 : NaN;
  snow.bal = valid ? bal : NaN;

  //Loop through angles attributes
  let angles_valid = {};
  const angles_attrs = Object.keys(angles);
  for (attr = 0; attr < angles_attrs.length; attr++) {
    angles_valid[angles_attrs[attr]] = valid ? angles[angles_attrs[attr]] : NaN;
  }
  return [
    sample_valid,
    angles_valid,
    snow
  ];
}

function aerosol_properties(height, cos_sa, aot = 0.1) {
  // Set an aerosol object to store the result. 
  let aerosol = {
    "tau": {},
    "p": {},
    "g": {},
    "gaer": {},
    "taumol": {},
    "tauaer": {}
  };

  // Loop through OLCI bands
  allBands.forEach(band => {
    tauaer = aot * Math.pow((wls[band] / 0.5), -1.3);
    g0 = 0.5263;
    g1 = 0.4627;
    wave0 = 0.4685;
    gaer = g0 + g1 * Math.exp(-wls[band] / wave0);
    pr = 0.75 * (1 + Math.pow(cos_sa, 2));
    taumol = Math.pow(wls[band], -4.05) * Math.min(1, Math.exp(-height / 7400)) * 0.00877;
    tau = tauaer + taumol;
    g = tauaer * gaer / tau;
    pa = (1 - Math.pow(g, 2)) / Math.pow((1 - 2 * g * cos_sa + Math.pow(g, 2)), 1.5);
    p = (taumol * pr + tauaer * pa) / tau;

    // Assign results to objects. Values can be called via, e.g., aerosol.tau.B01
    Object.assign(aerosol.tau, { [band]: tau })
    Object.assign(aerosol.p, { [band]: p })
    Object.assign(aerosol.g, { [band]: g })
    Object.assign(aerosol.gaer, { [band]: gaer })
    Object.assign(aerosol.taumol, { [band]: taumol })
    Object.assign(aerosol.tauaer, { [band]: tauaer })
  });
  return aerosol;
}

// Solar flux
function sol(x) {
  // SOLAR SPECTRUM at GROUND level
  // Inputs:
  // x - wave length in micrometer
  // Outputs: 
  // sol - solar spectrum in W m-2 micrometer-1 (?)
  sol1a = f0 * x;
  sol1b = - f1 * Math.exp(-bet * x) / bet;
  sol1c = - f2 * Math.exp(-gam * x) / gam;
  return sol1a + sol1b + sol1c;
}

function analyt_func(z1, z2) {
  // see BBA_calc_pol
  // compatible with array
  var gam2 = Math.exp(gam, 2);
  var gam3 = Math.exp(gam, 3);

  var z12 = Math.exp(z1, 2);
  var z13 = Math.exp(z1, 3);

  var z22 = Math.exp(z2, 2);
  var z23 = Math.exp(z2, 3);

  var bet2 = Math.exp(bet, 2);
  var bet3 = Math.exp(bet, 3);

  var ak1 = (z22 - z12) / 2.0;
  var ak2 = (z2 / bet + 1.0 / bet2) * Math.exp(-bet * z2) - (z1 / bet + 1.0 / bet2) * Math.exp(-bet * z1);
  var ak3 = (z2 / gam + 1.0 / gam2) * Math.exp(-gam * z2) - (z1 / gam + 1.0 / gam2) * Math.exp(-gam * z1);
  var am_minus = (z12 / bet + 2.0 * z1 / bet2 + 2.0 / bet3) * Math.exp(-bet * z1);

  var am1 = (z23 - z13) / 3.0;
  var am2 = (z22 / bet + 2.0 * z2 / bet2 + 2.0 / bet3) * Math.exp(-bet * z2) - am_minus
  var am3 = (z22 / gam + 2.0 * z2 / gam2 + 2.0 / gam3) * Math.exp(-gam * z2) - am_minus;

  return (f0 * ak1 - f1 * ak2 - f2 * ak3), (f0 * am1 - f1 * am2 - f2 * am3);
}

function prepare_coef(aerosol, angles) {
  // Set an atmosphere object 
  let atmosphere = {
    "t1": {},
    "t2": {},
    "ratm": {},
    "r": {}
  };

  // Loop through OLCI bands
  allBands.forEach(band => {
    tau = aerosol.tau[band];
    g = aerosol.g[band];
    p = aerosol.p[band];
    cos_sza = angles.cos_sza;
    cos_vza = angles.cos_vza;
    inv_cos_za = angles.inv_cos_za;

    one_g_tau = (1 - g) * tau;

    b1 = 1 + 1.5 * cos_sza + (1 - 1.5 * cos_sza) * Math.exp(-tau / cos_sza);
    b2 = 1 + 1.5 * cos_vza + (1 - 1.5 * cos_vza) * Math.exp(-tau / cos_vza);

    sumcos = cos_sza + cos_vza;

    astra = (1 - Math.exp(-tau * inv_cos_za)) / sumcos / 4;
    oskar = 4 + 3 * one_g_tau;

    rms = 1 - b1 * b2 / oskar + (3 * (1 + g) * (cos_sza * cos_vza) - 2 * sumcos) * astra;

    r = p * astra + rms;

    wa1 = 1.10363;
    wa2 = -6.70122;
    wx0 = 2.19777;
    wdx = 0.51656;
    bex = Math.exp((g - wx0) / wdx);

    arg = -0.5 * one_g_tau / ((wa1 - wa2) / (1 + bex) + wa2);

    t1 = Math.exp(arg / cos_sza);
    t2 = Math.exp(arg / cos_vza);

    a_s = [.18016, -0.18229, 0.15535, -0.14223];
    bs = [.58331, -0.50662, -0.09012, 0.0207];
    cs = [0.21475, -0.1, 0.13639, -0.21948];
    als = [0.16775, -0.06969, 0.08093, -0.08903];
    bets = [1.09188, 0.08994, 0.49647, -0.75218];

    a_cst = a_s[0] + a_s[1] * g;
    b_cst = bs[0] + bs[1] * g;
    c_cst = cs[0] + cs[1] * g;
    al_cst = als[0] + als[1] * g;
    bet_cst = bets[0] + bets[1] * g;

    for (num = 2; num < 4; num++) {
      if (num == 2) {
        gg = Math.pow(g, 2);
      } else {
        gg *= g;
      }
      a_cst += a_s[num] * gg
      b_cst += bs[num] * gg
      c_cst += cs[num] * gg
      al_cst += als[num] * gg
      bet_cst += bets[num] * gg
    }
    ratm = tau * (a_cst * Math.exp(-tau / al_cst) + b_cst * Math.exp(-tau / bet_cst) + c_cst)

    // Assign the results to the objects
    Object.assign(atmosphere.t1, { [band]: t1 });
    Object.assign(atmosphere.t2, { [band]: t2 });
    Object.assign(atmosphere.ratm, { [band]: ratm });
    Object.assign(atmosphere.r, { [band]: r });
  });

  return atmosphere;
}

function alb2rtoa(a, t1, t2, r0, ak1, ak2, ratm, r) {
  const surf = t1 * t2 * r0 * Math.pow(a, (ak1 * ak2 / r0)) / (1 - a * ratm);
  const rs = r + surf;
  return rs;
}

function clean_snow_albedo(sample_valid, angles, aerosol, atmosphere, snow) {
  snow.rs_1 = alb2rtoa(1, atmosphere.t1.B01, atmosphere.t2.B01, 1, 1, 1, atmosphere.ratm.B01, atmosphere.r.B01);
  snow.ind_clean = sample_valid.B01 >= snow.rs_1;
  snow.isnow = (snow.ind_clean) ? 0 : snow.isnow;
  snow.ind_pol = sample_valid.B01 < snow.rs_1;

  var alb_sph = Object.assign({}, emptyBandObject);
  allBands.forEach(band => {
    alb_sph[band] = Math.min(Math.exp(-Math.sqrt(1000 * 4 * Math.PI * (bai[band] / wls[band] * snow.al))), 1);
  });

  // Assign the results to the objects
  snow.alb_sph = alb_sph;
  return snow;
}

function polluted_snow_albedo(sample_valid, angles, aerosol, atmosphere, snow) {
  //throw new Error(JSON.stringify(sample_valid));

  if (snow.ind_pol) {
    snow.isnow = (snow.ind_pol) ? 1 : snow.isnow;
    ind_very_dark = (sample_valid.B21 < 0.4 && snow.ind_pol);
    snow.isnow = (ind_very_dark) ? 6 : snow.isnow;
    snow.r0 = (!ind_very_dark) ? snow.isnow : compute_rclean(angles.cos_sza, angles.cos_vza, angles.cos_sa, angles.raa);
    //snow.alb_sph = (snow.ind_pol) ? 1 : snow.alb_sph; Guess it applies to all bands
    Object.keys(snow.alb_sph).forEach(band => {
      snow.alb_sph[band] = (snow.ind_pol) ? 1 : snow.alb_sph[band]
    })

    const bands_to_loop_over = Object.keys(sample_valid);
    bands_to_loop_over.splice(18, 2);

    //throw new Error(JSON.stringify(bands_to_loop_over));

    for (i = 0; i < bands_to_loop_over.length; i++) {
      var band = bands_to_loop_over[i];
      snow.alb_sph[band] = (snow.ind_pol) ? solver_wrapper(sample_valid[band], atmosphere.t1[band], atmosphere.t2[band], snow.r0, angles.ak1, angles.ak2, atmosphere.ratm[band], atmosphere.r[band]) : snow.alb_sph[band];
      snow.isnow = snow.alb_sph[band] == -999 ? -(i + 1) : snow.isnow;
    }

    // Guess it applies to all bands
    Object.keys(snow.alb_sph).forEach(band => {
      snow.alb_sph[band] = snow.isnow ? snow.alb_sph[band] : NaN;
    })

    let ind_clear_pol = (snow.alb_sph.B01 > 0.98 | snow.alb_sph.B02 > 0.98) & snow.ind_pol;
    snow.isnow = ind_clear_pol ? 7 : snow.isnow;

    // Guess it applies to all bands
    if (!ind_clear_pol) {
      allBands.forEach(band => {
        snow.alb_sph[band] = Math.exp(-Math.sqrt(4 * 1000 * Math.PI * snow.al * (bai[band] / wls[band])));
      });
    }

    snow.ind_pol = snow.ind_pol & (snow.isnow != 7);

    // reprocessing of albedo to remove gaseous absorption using linear polynomial approximation in the range 753-778nm.
    // Meaning: alb_sph[12],alb_sph[13] and alb_sph[14] are replaced by a linear  interpolation between alb_sph[11] and alb_sph[15]
    if (ind_clear_pol) {
      afirn = snow.alb_sph['B15'] - snow.alb_sph['B11'] / (wls['B15'] - wls['B11']);
      bfirn = snow.alb_sph['B15'] - afirn * wls['B15'];

      // indeces of bands start with 0!
      allBands.concat().splice(11, 3).forEach(band => {
        snow.alb_sph[band] = bfirn + afirn * wls[band];
      });
    }

    // pixels that are clean enough in channels 18 19 20 and 21 are not affected by pollution, the analytical equation can then be used
    ind_ok = (sample_valid.B20 > 0.35) & snow.ind_pol;
    if (ind_ok) {
      allBands.concat().splice(17, 4).forEach(band => {
        snow.alb_sph[band] = Math.exp(-Math.sqrt(4. * 1000. * Math.PI * snow.al * (bai[band] / wls[band])));
      });
    }

    //to avoid the influence of gaseous absorption (water vapor) we linearly interpolate in the range 885-1020nm for bare ice cases only (low toa[20])
    //Meaning: alb_sph[18] and alb_sph[19] are replaced by a linear interpolation between alb_sph[17] and alb_sph[20]
    if (ind_ok) {
      bcoef = (snow.alb_sph['B20'] - snow.alb_sph['B17']) / (wls['B20'] - wls['17'])
      acoef = snow.alb_sph['B20'] - bcoef * wls['B20']
      allBands.concat().splice(17, 2).forEach(band => {
        snow.alb_sph[band] = acoef + bcoef * wls[band];
      });
    }
  }
  return snow;
}

function compute_plane_albedo(snow, angles) {
  var rp = Object.assign({}, emptyBandObject);
  var refl = Object.assign({}, emptyBandObject);

  snow.rp = rp;
  snow.refl = refl;

  allBands.forEach(band => {
    snow.rp[band] = Math.pow(snow.alb_sph[band], angles.ak1);
    snow.refl[band] = snow.r0[band] * Math.pow(snow.alb_sph[band], angles.ak1 * angles.ak2 / snow.r0);
  });

  ind_all_clean = snow.ind_clean || (snow.isnow == 7);


  if (ind_all_clean) {
    snow.rp3 = plane_albedo_sw_approx(snow.diameter, angles.cos_sza);
    snow.rs3 = spher_albedo_sw_approx(snow.diameter);
  }

  // solar flux calculation
  // sol1      visible(0.3 - 0.7micron)
  // somehow, a different sol1 needs to be used for clean snow and polluted snow
  var sol1_pol = sol(0.7) - sol(0.3);
  // sol2      near - infrared(0.7 - 2.4micron)
  // same for clean and polluted
  var sol2 = sol(2.4) - sol(0.7);

  // sol3      shortwave(0.3 - 2.4 micron)
  // sol3 is also different for clean snow and polluted snow
  var sol3_pol = sol1_pol + sol2;

  // asol specific band
  var asol = sol(0.865) - sol(0.7);

  if (snow.ind_pol) {
    snow.rp3 = BBA_calc_pol(snow.rp, asol, sol1_pol, sol3_pol);
    snow.rs3 = BBA_calc_pol(snow.alb_sph, asol, sol1_pol, sol3_pol);
  }

  return snow;
}

function BBA_calc_pol(alb, asol, sol1_pol, sol3_pol) {
  // polluted snow
  // NEW CODE FOR BBA OF BARE ICE
  // alb is either the planar or spherical albedo

  // ANAlYTICal EQUATION FOR THE NOMINATOR
  // integration over 3 segments

  // segment 1
  // QUADRATIC POLYNOMIal for the range 400 - 709nm
  // input wavelength
  //    alam2 = w[0]
  //    alam3 = w[5]
  //    alam5 = w[10]
  //    alam6 = w[11]
  //    alam7 = w[16]
  //    alam8 = w[20]

  const alam2 = 0.4;
  const alam3 = 0.56;
  const alam5 = 0.709;
  const alam6 = 0.753;
  const alam7 = 0.865;
  const alam8 = 1.02;

  // input reflectances
  var r2 = alb['B01'];
  var r3 = alb['B05'];
  var r5 = alb['B10'];
  var r6 = alb['B011'];
  var r7 = alb['B16'];
  var r8 = alb['B20'];

  // declared outside
  //var coef1, coef2 = analyt_func(0.3, 0.7);
  //var coef3, coef4 = analyt_func(0.7, 0.865);

  var a1, b1, c1 = quad_func(alam2, alam3, alam5, r2, r3, r5)
  var aj1 = a1 * sol1_pol + b1 * coef1 + c1 * coef2;

  // segment 2.1
  // QUADRATIC POLYNOMIal for the range 709 - 865nm
  var a2, b2, c2 = quad_func(alam5, alam6, alam7, r5, r6, r7)
  var aj2 = a2 * asol + b2 * coef3 + c2 * coef4;    // segment 2.2

  // exponential approximation for the range 865 - 2400 nm
  const z1 = 0.865;
  const z2 = 2.4;
  var rati = r7 / r8;
  var alasta = (alam8 - alam7) / Math.log(rati);
  var an = 1. / alasta;
  var p = r7 * Math.exp(alam7 / alasta);

  var aj31 = (1. / an) * (Math.exp(-an * z2) - Math.exp(-an * z1));
  var aj32 = (1. / (bet + an)) * (Math.exp(-(bet + an) * z2) - Math.exp(-(an + bet) * z1));
  var aj33 = (1. / (gam + an)) * (Math.exp(-(gam + an) * z2) - Math.exp(-(an + gam) * z1));
  var aj3 = (-f0 * aj31 - f1 * aj32 - f2 * aj33) * p;

  return (aj1 + aj2 + aj3) / sol3_pol;
}

function quad_func(x0, x1, x2, y0, y1, y2) {
  // quadratic function used for the polluted snow BBA calculation
  // see BBA_calc_pol
  // compatible with arrays
  var d1 = (x0 - x1) * (x0 - x2);
  var d2 = (x1 - x0) * (x1 - x2);
  var d3 = (x2 - x0) * (x2 - x1);

  var a1 = x1 * x2 * y0 / d1 + x0 * x2 * y1 / d2 + x0 * x1 * y2 / d3;
  var b1 = -(x1 + x2) * y0 / d1 - (x0 + x2) * y1 / d2 - (x0 + x1) * y2 / d3;
  var c1 = y0 / d1 + y1 / d2 + y2 / d3;
  var x = x1;
  return a1, b1, c1;
}

function spher_albedo_sw_approx(D) {
  return 0.6420 + 0.1044 * Math.exp(-1000 * D / 158.62) + 0.1773 * Math.exp(-1000 * D / 2448.18);
}

function plane_albedo_sw_approx(D, cos_sza) {
  var cos_sza2 = Math.exp(cos_sza, 2);
  var anka = 0.7389 - 0.1783 * cos_sza + 0.0484 * cos_sza2;
  var banka = 0.0853 + 0.0414 * cos_sza - 0.0127 * cos_sza2
  var canka = 0.1384 + 0.0762 * cos_sza - 0.0268 * cos_sza2;
  var diam1 = 187.89 - 69.2636 * cos_sza + 40.4821 * cos_sza2;
  var diam2 = 2687.25 - 405.09 * cos_sza + 94.5 * cos_sza2;
  return anka + banka * Math.exp(-1000 * D / diam1) + canka * Math.exp(-1000 * D / diam2)
}

function compute_rclean(cos_sza, cos_vza, cos_sa, raa) {
  const am11 = Math.sqrt(1 - Math.pow(cos_sza, 2));
  const am12 = Math.sqrt(1 - Math.pow(cos_vza, 2));
  const theta = Math.acos(-cos_sza * cos_vza + am11 * am12 * Math.cos(raa * 3.14159 / 180)) * 180 / Math.PI;
  const pz = 11.1 * Math.exp(-0.087 * theta) + 1.1 * Math.exp(-0.014 * theta);
  const sumcos = cos_sza + cos_vza;
  const rclean = 1.247 + 1.186 * sumcos + 5.157 * cos_sza * cos_vza + pz;
  return rclean / 4 / sumcos;
}

function solver_wrapper(toa_cor_o3, t1, t2, r0, ak1, ak2, ratm, r) {
  return zbrent(0.1, 1, args = [t1, t2, r0, ak1, ak2, ratm, r, toa_cor_o3], max_iter = 30, tolerance = 2e-4);
}

function zbrent(x0, x1, args, max_iter = 100, tolerance = 1e-6) {
  let fx0 = f(x0, ...args = args);
  let fx1 = f(x1, ...args = args);
  if ((fx0 * fx1) > 0) {
    return -999;
  }
  if (Math.abs(fx0) < Math.abs(fx1)) {
    [x0, x1] = [x1, x0];
    [fx0, fx1] = [fx1, fx0];
  }
  let [x2, fx2] = [x0, fx0];
  let d = x2;
  let mflag = true;
  let steps_taken = 0;
  while (steps_taken < max_iter && Math.abs(x1 - x0) > tolerance) {
    fx0 = f(x0, ...args = args);
    fx1 = f(x1, ...args = args);
    fx2 = f(x2, ...args = args);

    if (fx0 != fx2 && fx1 != fx2) {
      let L0 = (x0 * fx1 * fx2) / ((fx0 - fx1) * (fx0 - fx2));
      let L1 = (x1 * fx0 * fx2) / ((fx1 - fx0) * (fx1 - fx2));
      let L2 = (x2 * fx1 * fx0) / ((fx2 - fx0) * (fx2 - fx1));
      var new_value = L0 + L1 + L2;
    } else {
      var new_value = x1 - ((fx1 * (x1 - x0)) / (fx1 - fx0));
    }

    if (
      (new_value < ((3 * x0 + x1) / 4) || new_value > x1) ||
      (mflag == true && (Math.abs(new_value - x1)) >= (Math.abs(x1 - x2) / 2)) ||
      (mflag == false && (Math.abs(new_value - x1)) >= (Math.abs(x2 - d) / 2)) ||
      (mflag == true && (Math.abs(x1 - x2)) < tolerance) ||
      (mflag == false && (Math.abs(x2 - d)) < tolerance)
    ) {
      new_value = (x0 + x1) / 2;
      mflag = true;
    } else {
      mflag = false;
    }

    let fnew = f(new_value, ...args = args);
    [d, x2] = [x2, x1];

    if (fx0 * fnew < 0) {
      x1 = new_value;
    } else {
      x0 = new_value;
    }

    if (Math.abs(fx0) < Math.abs(fx1)) {
      [x0, x1] = [x1, x0];
    }

    steps_taken += 1;
  }
  return x1;
}

// not used?
function f(albedo, t1, t2, r0, ak1, ak2, ratm, r, toa_cor_o3) {
  const surf = t1 * t2 * r0 * albedo ** (ak1 * ak2 / r0) / (1 - albedo * ratm);
  const rs = r + surf
  return toa_cor_o3 - rs;
}
"""

logging.info("Configuration ok.")    

2023-03-07 14:49:11 [INFO] root - Configuration ok.


In [8]:
date_range = (f'{date}T8:00:00', f'{date}T18:00:00')

DATE_FOLDER = date.replace("-","_")
DL_FOLDER =  os.path.join('downloads', str(resolution), DATE_FOLDER)
PROCESSED_FOLDER = f'{DL_FOLDER}/processed'

DATE_FOLDER

'2021_07_09'

In [9]:
aoi_gpd = gpd.GeoDataFrame(geometry=[wkt.loads(aoi)], crs = "EPSG:4326")

In [12]:
# load the 1 degree world grid from which the tiles will be calculated
logging.info("Loading grid")
grid = gpd.read_file(os.environ["DATA_PATH"] + "/wgs84-1degree.geojson")

2023-03-07 14:49:35 [INFO] root - Loading grid


In [13]:
logging.info("Calculating tiles to be processed.")    

# make the intersection with the polygon so that we only calculate tiles inside the polygon
toProcess=gpd.overlay(aoi_gpd, grid, how='intersection')

#Remove chunks that are too small to be processed
MIN_AREA = 1e-1
toProcess = toProcess[toProcess.geometry.to_crs('epsg:4326').area > MIN_AREA]

2023-03-07 14:49:39 [INFO] root - Calculating tiles to be processed.
/tmp/ipykernel_386/667727171.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  toProcess = toProcess[toProcess.geometry.to_crs('epsg:4326').area > MIN_AREA]


In [14]:
# use full frames
toProcess = grid[grid.ID.isin(list(toProcess.ID.values))]

In [15]:
if DELETE_DOWNLOAD_FOLDER and os.path.exists(DL_FOLDER):
    shutil.rmtree(DL_FOLDER)
    logging.info(f'Folder {DL_FOLDER} deleted')

2023-03-07 14:49:46 [INFO] root - Folder downloads/1200/2021_07_09 deleted


In [16]:
listBbox = [BBox(bbox=geom, crs="EPSG:4326") for geom in toProcess.geometry]
listSizes = [bbox_to_dimensions(bbox, resolution=resolution) for bbox in listBbox]
folderPaths = [f'{DL_FOLDER}/{round(id)}' for id in toProcess.ID]

/home/88c8cc07-d79c-45b9-a407-ba18967711b0/.local/lib/python3.9/site-packages/sentinelhub/geometry.py:113: SHDeprecationWarning: Initializing `BBox` objects from `shapely` geometries will no longer be possible in future versions. Use the `bounds` property of the `shapely` geometry to initialize the `BBox` instead.
  x_fst, y_fst, x_snd, y_snd = BBox._to_tuple(bbox)


In [17]:
requests = [SentinelHubRequest(
            evalscript=evalscript,
            data_folder=folderPath,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.DEM_COPERNICUS_30,
                    identifier="COP_30",
                    upsampling="NEAREST",
                    downsampling="NEAREST",
                ),
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL3_OLCI,
                    identifier="OLCI",
                    time_interval=date_range,
                    upsampling="NEAREST",
                    downsampling="NEAREST",
                ),
            ],
            responses=[
                SentinelHubRequest.output_response('r_TOA_01', MimeType.TIFF),
                SentinelHubRequest.output_response('r_TOA_06', MimeType.TIFF),
                SentinelHubRequest.output_response('r_TOA_17', MimeType.TIFF),
                SentinelHubRequest.output_response('r_TOA_21', MimeType.TIFF),
                SentinelHubRequest.output_response('snow_grain_diameter', MimeType.TIFF),
                SentinelHubRequest.output_response('snow_specific_surface_area', MimeType.TIFF),
                SentinelHubRequest.output_response('diagnostic_retrieval', MimeType.TIFF),
                SentinelHubRequest.output_response('albedo_bb_planar_sw', MimeType.TIFF),
                SentinelHubRequest.output_response('albedo_bb_spherical_sw', MimeType.TIFF),
            ],
            bbox=bb,
            size=size,
            config=sh_config
        ) for bb, size, folderPath in zip(listBbox, listSizes, folderPaths)]

#extract only downloader
list_of_requests = [request.download_list[0] for request in requests]

In [18]:
logging.info('Starting download')
downloader = SentinelHubDownloadClient(config=sh_config)
downloader.download(list_of_requests, max_threads=20, show_progress=False)
logging.info('Done')

2023-03-07 14:49:46 [INFO] root - Starting download
2023-03-07 14:49:50 [INFO] root - Done


In [19]:
def unzipFile(file, destination):
    if not os.path.exists(os.path.join(destination, 'snow_grain_diameter.tif')):
        tar = tarfile.open(file, "r:")
        tar.extractall(path=destination)
        tar.close()

In [20]:
class ConcurrentUnzipper:
    def __init__(self, files, destinations):
        self.files = files
        self.destinations = destinations
        
    def operation(self, chunk):
        unzipFile(self.files[chunk] , self.destinations[chunk])
        
    def unzipAll(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            list(executor.map(self.operation, np.arange(0, len(self.files)), chunksize=1))

In [21]:
filenamesList = glob.glob(f'./{DL_FOLDER}/*/*/response.tar')
destinations =  [file.replace('response.tar', '') for file in filenamesList]        
        
logging.info('Extracting .tar responses')
unzipper = ConcurrentUnzipper(filenamesList, destinations)
unzipper.unzipAll()
logging.info('Done')

2023-03-07 14:49:50 [INFO] root - Extracting .tar responses
2023-03-07 14:49:50 [INFO] root - Done


In [22]:
def mergeProduct(productName):
    prodResult = productName.replace(".tif", '_merged.tif')
    if os.path.exists(prodResult):
        os.remove(prodResult)
        logging.info(f'Deleted file: {prodResult}')
    filenamesList = glob.glob(f'./{DL_FOLDER}/*/*/{productName}')
    merge_tiffs(filenamesList, prodResult, overwrite=True)
     
class TifMerger:
    def __init__(self, products):
        self.products = products
        
    def operation(self, chunk):
        mergeProduct(self.products[chunk])
        
    def process(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            list(executor.map(self.operation, np.arange(0, len(self.products)), chunksize=1))    

In [23]:
#define the products that will be computed
products = ['diagnostic_retrieval.tif','albedo_bb_planar_sw.tif','albedo_bb_spherical_sw.tif','snow_grain_diameter.tif', 'snow_specific_surface_area.tif', 'r_TOA_01.tif', 'r_TOA_06.tif', 'r_TOA_17.tif', 'r_TOA_21.tif']
#products = ['snow_grain_diameter.tif']

#merge individual tiles into one single image
logging.info('Merging products')
unzipper = TifMerger(products)
unzipper.process()
logging.info('Done')    

resultsDataPath = os.path.join(USR_PATH, PROCESSED_FOLDER)

2023-03-07 14:49:50 [INFO] root - Merging products
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to albedo_bb_planar_sw_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to r_TOA_17_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to diagnostic_retrieval_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to r_TOA_06_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to albedo_bb_spherical_sw_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to snow_specific_surface_area_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to snow_grain_diameter_merged.tif
2023-03-07 14:49:50 [INFO] root - merging 4 tiffs to r_TOA_01_merged.tif


00000000

2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging 4 tiffs to r_TOA_21_merged.tif
2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging done


...10...20.....10...20...30...40...50...60...70.....10...20.....10...20...30...40...50...10...20.....60...70.....10...20.....10...20...80...90...100 - done.
...10...20...30...40...50.30...40...50...60...70.....60...70...30...40...50...60...70...80...90...100 - done.
.30...40...50.30...40...50...60...70.....60...70...30...40...50...60...70...80...90...100 - done.
.80...90...100 - done.
.80...90...100 - done.
.80...90...100 - done.
.80...90...100 - done.
.80...90...100 - done.


2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - merging done


0

2023-03-07 14:49:52 [INFO] root - merging done
2023-03-07 14:49:52 [INFO] root - Done


...10...20...30...40...50...60...70...80...90...100 - done.


In [24]:
if not os.path.exists(PROCESSED_FOLDER):
    os.makedirs(PROCESSED_FOLDER)
    
producedFiles = glob.glob('*_merged.tif')
for prodResult in producedFiles:
    shutil.move(prodResult, f'{PROCESSED_FOLDER}/{prodResult}')
    
resultsDataPath = os.path.join(USR_PATH, PROCESSED_FOLDER)
resultsDataPath

'/home/88c8cc07-d79c-45b9-a407-ba18967711b0/downloads/1200/2021_07_09/processed'

In [25]:
processedDataPath = os.path.join(USR_PATH, PROCESSED_FOLDER)
if len(projection) > 0 and not projection == '4326' : 
    destFolder = f'{DL_FOLDER}/warped/'

    logging.info('Reprojecting to epsg:{projection}')
    
    if os.path.exists(destFolder):
        shutil.rmtree(destFolder)

    os.makedirs(destFolder)

    for product in products:
        srcFile = processedDataPath + "/" + product.replace(".tif", "_merged.tif")
        destFile = f'{destFolder}/{product}'
        cmd = f'gdalwarp {srcFile} {destFile}  -t_srs EPSG:{projection}'
        os.system(cmd)
   
    resultsDataPath = destFolder
else:
    for product in products:
        srcFile = processedDataPath + "/" + product.replace(".tif", "_merged.tif")
        destFile = processedDataPath + "/" + product
        if os.path.exists(srcFile):
            os.rename(srcFile, destFile)